In [2]:
from __future__ import print_function
import numpy

from nltk.stem import WordNetLemmatizer
import tempfile
import string

import sys
import logging
import os
import pickle
import re

# Using Gensim

In [3]:
from glob import glob
import re
import string
import funcy as fp
from gensim import models
from gensim.corpora import Dictionary, MmCorpus
import nltk
import pandas as pd


In [4]:
import logging
logging.basicConfig(filename='gensim.log',
                    format="%(asctime)s:%(levelname)s:%(message)s",
                    level=logging.INFO)

In [5]:
# load the data

filename = 'df_diag.pkl'
response_folder = '~/Documents/Metis/project4'
response_folder = os.path.expanduser(response_folder)

with open(os.path.join(response_folder, filename), 'rb') as f:
    df = pickle.load(f)
df.columns = ['diag']

In [6]:
# data cleaning separate the staging direction with conversation
def sepExp(text):
    regex = re.compile(".*?\((.*?)\)")
    expression = re.findall(regex, text)
    expression = ','.join(expression)
    diag_filt = re.sub("[\(\[].*?[\)\]]", "", text)

    return expression, diag_filt

In [7]:
df['expression'], df['diag_filt'] = zip(*df['diag'].map(sepExp))

In [8]:
import nltk
from nltk.corpus import stopwords
from IPython.core.debugger import set_trace

nltk.download('wordnet')
stop = set(stopwords.words('english'))

def process(text):
    text = text.lower()
    text_filt = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    text_filt_le = [wordnet_lemmatizer.lemmatize(i) for i in text_filt]
    words_filt = [w for w in text_filt_le if w not in stop]
    return words_filt

df['tokens'] = df['diag_filt'].apply(process)
print(df.tokens)

[nltk_data] Downloading package wordnet to /Users/xuanqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
phoebe:      [wait, doe, eat, chalk, cause, want, went, car...
rachel:      [god, monica, thank, god, went, building, guy,...
ross:        [feel, like, someone, reached, throat, grabbed...
monica:      [nothing, tell, guy, work, okay, everybody, re...
chandler:    [right, joey, nice, doe, hump, hump, hairpiece...
joey:        [mon, going, guy, gotta, something, wrong, ins...
Name: tokens, dtype: object


In [9]:
# prepare the corpus

def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=3, no_above=0.5):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
#     print(dictionary)
    stopwords = nltk_stopwords().union(additional_stopwords)
#     print(stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
#     print(stopword_ids)
    dictionary.filter_tokens(stopword_ids)
#     print(dictionary)
    dictionary.compactify()
#     print(dictionary)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    print(dictionary)
    dictionary.compactify()
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    return dictionary, corpus

In [10]:
dictionary, corpus = prep_corpus(df.tokens)


Building dictionary...
Dictionary(1171 unique tokens: ['accepted', 'accomplished', 'adopted', 'adventure', 'agrees']...)
Building corpus...


In [11]:
MmCorpus.serialize('friends.mm', corpus)
dictionary.save('friends.dict')

In [12]:
# %%time
# lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=50)
                                      
# lda.save('friends.model')

CPU times: user 1.14 s, sys: 118 ms, total: 1.26 s
Wall time: 1.14 s


In [82]:
lda_model = models.ldamodel.LdaModel(corpus=corpus,
         id2word=dictionary,
         num_topics=10,
         eval_every=10,
         passes=50,
         iterations=5000,
         random_state=numpy.random.RandomState(15))



In [46]:
# convergence test

import re
import matplotlib.pyplot as plt
p = re.compile("(-*\d+\.\d+) per-word .* (\d+\.\d+) perplexity")
matches = [p.findall(l) for l in open('gensim.log')]
matches = [m for m in matches if len(m) > 0]
tuples = [t[0] for t in matches]
perplexity = [float(t[1]) for t in tuples]
liklihood = [float(t[0]) for t in tuples]
iter = list(range(0,len(tuples)*10,10))
plt.plot(iter,liklihood,c="black")
plt.ylabel("log liklihood")
plt.xlabel("iteration")
plt.title("Topic Model Convergence")
plt.grid()
plt.savefig("convergence_liklihood.pdf")
plt.close()

<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: DeprecationWarning: invalid escape sequence \d
<input>:3: Dep

In [18]:
import pyLDAvis.gensim as gensimvis
import pyLDAvis

In [83]:
vis_data1 = gensimvis.prepare(lda_model, corpus, dictionary)
# gensimvis.prepare(lda_model, corpus, dictionary)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [84]:
pyLDAvis.display(vis_data1)

In [88]:
# save the results to html file
friend_lda = open(os.path.join(response_folder, 'friend_lda.html'), 'w')
pyLDAvis.save_html(vis_data,friend_lda)

In [89]:
%%time
# The optional parameter T here indicates that HDP should find no more than 50 topics
# if there exists any.
hdp = models.hdpmodel.HdpModel(corpus, dictionary, T=50)
hdp.save('newsgroups_hdp.model')


CPU times: user 66.3 ms, sys: 13.7 ms, total: 80 ms
Wall time: 72 ms


In [90]:
vis_data = gensimvis.prepare(hdp, corpus, dictionary)
pyLDAvis.display(vis_data)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [381]:

# dct = Dictionary(df.tokens)  # fit dictionary

# doc1 = dct.doc2bow(df.tokens[1])

In [397]:
# lda.get_document_topics(doc1,per_word_topics=True)

In [277]:
dictionary_0 = Dictionary(df.tokens)

# Add season in this content

In [102]:
import os
from os import listdir
from os.path import isfile, join
import glob

response_folder = '~/Documents/Metis/project4'
response_folder = os.path.expanduser(response_folder)
comp_files = [f for f in listdir(response_folder) if isfile(join(response_folder, f))]

df_season=pd.DataFrame()
count=0
for filename in glob.glob(os.path.join(response_folder,'chapter*.pkl')):
    #print(filename)
    with open(filename,'rb') as f:
        df_temp = pickle.load(f)
        
        df_season = df_season.append(df_temp.loc['rachel:'])
        #print(len(df_movie))

In [103]:
df_season.reset_index(inplace=True)
df_season.set_index('season',inplace=True)

In [104]:
def sepExp(text):
    regex = re.compile(".*?\((.*?)\)")
    expression = re.findall(regex, text)
    expression = ','.join(expression)
    diag_filt = re.sub("[\(\[].*?[\)\]]", "", text)

    return expression, diag_filt


<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:2: DeprecationWarning: invalid escape sequence \(
<input>:5: DeprecationWarning: invalid escape sequence \(
<input>:2: Dep

In [105]:
df_season['expression'], df_season['diag_filt'] = zip(*df_season['diag'].map(sepExp))

In [106]:
import nltk
from nltk.corpus import stopwords
from IPython.core.debugger import set_trace

nltk.download('wordnet')
stop = set(stopwords.words('english'))

def process(text):
    text = text.lower()
    text_filt = re.findall(r'\b[a-zA-Z]{3,}\b', text)
    
    wordnet_lemmatizer = WordNetLemmatizer()
    text_filt_le = [wordnet_lemmatizer.lemmatize(i) for i in text_filt]
    words_filt = [w for w in text_filt_le if w not in stop]
    return words_filt
df_season['tokens'] = df_season['diag_filt'].apply(process)
print(df_season.tokens)

[nltk_data] Downloading package wordnet to /Users/xuanqi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
season
4.0     [nooo, way, romantic, song, ever, way, song, w...
3.0     [god, god, excuse, move, move, emergency, excu...
2.0     [god, monica, thank, god, went, building, guy,...
6.0     [said, rachel, right, think, mon, honey, gotta...
5.0     [welcome, million, god, whoa, well, tell, mood...
7.0     [well, hello, rachel, wait, okay, monica, need...
8.0     [yeah, pheebs, honey, got, engaged, couple, ho...
10.0    [joey, god, okay, hall, rachel, room, rachel, ...
9.0     [god, thank, deal, quite, yet, said, wa, disag...
11.0    [ooh, lovebarbados, rachel, kissing, know, ooh...
Name: tokens, dtype: object


In [107]:
df_season.head()

,index,diag,expression,diag_filt,tokens
season,,,,,
4.0,rachel:,Nooo Way! The most romantic song ever is The ...,"she leans over and kisses him on the cheek,ent...",Nooo Way! The most romantic song ever is The ...,"[nooo, way, romantic, song, ever, way, song, w..."
3.0,rachel:,Oh my god. Oh my god.(She decides to make a b...,"She decides to make a break for it.,She tries ...","Oh my god. Oh my god.,Excuse me. Move! Move! ...","[god, god, excuse, move, move, emergency, excu..."
2.0,rachel:,Oh God Monica hi! Thank God! I just went to y...,"to,the waitress that brought her coffee,to Mon...",Oh God Monica hi! Thank God! I just went to y...,"[god, monica, thank, god, went, building, guy,..."
6.0,rachel:,(To the woman sitting in front of her) He-he s...,"To the woman sitting in front of her,laughs,Th...","He-he said Rachel, right? Do,you think I shou...","[said, rachel, right, think, mon, honey, gotta..."
5.0,rachel:,"Oh, you’re welcome a million.,(softly) Oh my ...","softly,interrupting,handing him the letter,com...","Oh, you’re welcome a million., Oh my God., Wh...","[welcome, million, god, whoa, well, tell, mood..."


In [109]:
def nltk_stopwords():
    return set(nltk.corpus.stopwords.words('english'))

def prep_corpus(docs, additional_stopwords=set(), no_below=2, no_above=0.4):
    print('Building dictionary...')
    dictionary = Dictionary(docs)
#     print(dictionary)
    stopwords = nltk_stopwords().union(additional_stopwords)
#     print(stopwords)
    stopword_ids = map(dictionary.token2id.get, stopwords)
#     print(stopword_ids)
    dictionary.filter_tokens(stopword_ids)
#     print(dictionary)
    dictionary.compactify()
#     print(dictionary)
    dictionary.filter_extremes(no_below=no_below, no_above=no_above, keep_n=None)
    print(dictionary)
    dictionary.compactify()
    
    print('Building corpus...')
    corpus = [dictionary.doc2bow(doc) for doc in docs]
    
    return dictionary, corpus

In [111]:
dictionary, corpus = prep_corpus(df_season.tokens)

Building dictionary...
Dictionary(1301 unique tokens: ['accidentally', 'acting', 'admit', 'adorable', 'agent']...)
Building corpus...


In [112]:
MmCorpus.serialize('rachel_season.mm', corpus)
dictionary.save('rachel_season.dict')

In [113]:
%%time
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, passes=50)
                                      
lda.save('rachel_season.model')

CPU times: user 1.17 s, sys: 68.5 ms, total: 1.24 s
Wall time: 1.17 s


In [116]:
vis_data = gensimvis.prepare(lda, corpus, dictionary)

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


In [115]:
pyLDAvis.display(vis_data)

In [581]:
rachel_lda = open(os.path.join(response_folder, 'rachel_lda.html'), 'w')
pyLDAvis.save_html(vis_data,rachel_lda)